In [11]:
import numpy as np
import time
import json
from generalUtilities.ModMunkres import ModMunkres2
from generalUtilities.Kepler import Kepler
from generalUtilities.mtspofs_ga import mtspofs_ga
from generalUtilities.utilities import zipArray
from generalUtilities.Classes import *
from generalUtilities.config import *

# MunkresPairing Testbed

In [ ]:
divertMat = np.random.randint(3, size=(6,9))
threatVal = np.random.rand(9)
costMat = np.random.rand(6, 9)
costMat[np.random.randint(costMat.shape[0])] = np.zeros(costMat.shape[1])
costMat[:, np.random.randint(costMat.shape[1])] = np.zeros(costMat.shape[0])
allowedAssigns = np.ceil(costMat)
adjustedCost = costMat - 9.9e-3
adjustedCost[ (adjustedCost < 0) | (adjustedCost == -0) ] = 0

available = np.ceil( adjustedCost/np.amax(adjustedCost) )

print("initial availability matrix: \n{}".format(available))

if not np.any(available):
    available = np.ones_like(costMat)
# find rows and columns that are empty
availableRows = np.any(available, 1)
emptyRows = np.asscalar(np.where(~np.any(available, 1))[0])
availableCols = np.any(available, 0)
emptyCols = np.asscalar(np.where(~np.any(available, 0))[0])
unAssignedKVs = np.where(availableRows == 0)[0]
firstCut = costMat[availableRows, :]
finalCut = firstCut[:, availableCols]

redAllowed = allowedAssigns[availableRows, :]
allowed = redAllowed[:, availableCols]

_, _, _, assignTemp = ModMunkres2(finalCut, mask=int)
# adjust assign by allowed assignments matrix
assign = assignTemp * allowed

print("\ndivert matrix: \n{}".format(divertMat))
print("\noriginal costMat: \n{}".format(costMat))
print("\noriginal availablility: {}".format(np.any(available)))
print("emptyRows: {}, emptyCols: {}".format(emptyRows, emptyCols))
print("\nunAssignedKVs: {}".format(unAssignedKVs))
print("\nfinal allowed assignments matrix: \n{}".format(allowed))
print("\nassignments from ModMunkres2: \n{}".format(assignTemp))
print("\nadjusted assignments: \n{}".format(assign))Mu

In [ ]:
assign = np.insert(assign, emptyRows, np.zeros(assign.shape[1]), 0)

assign = np.insert(assign, emptyCols, np.zeros(assign.shape[0]), 1)

unAssignedThreats = np.where(~np.any(assign, 0))[0]

print("final assignment matrix: \n{}".format(assign))
print("\nunAssignedThreats: {}".format(unAssignedThreats))
print("final assignment shape verification: {} to {}".format(assign.shape, costMat.shape))

In [ ]:
disallowedThreats = np.where(~np.any(allowed, 0))[0]
availableThreats = np.setdiff1d(unAssignedThreats, disallowedThreats)

print("disallowed Threats: {}".format(disallowedThreats))
print("available Threats: {}".format(availableThreats))

In [ ]:
minCost = np.finfo(float).max
for kvID in unAssignedKVs:
    currentAssign = -1*np.ones(2)
    for threatID in availableThreats:
        if adjustedCost[kvID, threatID] < minCost:
            minCost = adjustedCost[kvID, threatID]
            currentAssign[:] = [kvID, threatID]
            print("currentAssign selection: {}".format(currentAssign))
    # assign the kv to the id with the lowest cost
    assign[int(currentAssign[0]), int(currentAssign[1])] = 1
    disallowedThreats = np.append(disallowedThreats, currentAssign[1])
    availableThreats = np.setdiff1d(unAssignedThreats, disallowedThreats)
    print("updated availableThreats list: {}".format(availableThreats))
print("updated final assignment matrix: \n{}".format(assign))
print("total number of assignments: {}".format(len(np.where(assign==1)[0])))

In [ ]:
nWeapons, nThreats = assign.shape

indices = zipArray(np.where(assign==1))

wpnIndices = indices[:,0]
tgtIndices = indices[:,1]

print("seleced indices: \n{}".format(indices))
print("wpnIndices: \n{}".format(wpnIndices))
print("tgtIndices: \n{}".format(tgtIndices))

In [ ]:
# assign = originalAssignment.copy()
originalAssignment = assign.copy()
for iwpn in range(wpnIndices.size):
    for itgt in range(tgtIndices.size):
        for jwpn in range(wpnIndices.size):
            for jtgt in range(tgtIndices.size):
                if iwpn == jwpn and itgt == jtgt:
                    continue
                if assign[iwpn, itgt] == 1 and assign[jwpn, jtgt] == 1:
                    if divertMat[iwpn, jtgt] < divertMat[jwpn, itgt] and divertMat[iwpn, jtgt] > 0:
                        assign[iwpn, itgt] = 0
                        assign[iwpn, jtgt] = 1
                        assign[jwpn, jtgt] = 0
                        assign[jwpn, itgt] = 1
                        
# should be done with minimizing divert:
print("old assignment matrix: \n{}".format(originalAssignment))
print("new assignment matrix: \n{}".format(assign))
print("total number of assignments: {}".format(len(np.where(assign==1)[0])))                        

In [ ]:
targetedThreats = np.where(assign == 1)[1]
avgPk = np.random.rand()

originalVal = threatVal.copy()

threatVal[targetedThreats] = threatVal[targetedThreats] * (1.0 - avgPk)

bigDeal = np.argmax(threatVal)
bigLeth = np.amax(threatVal)

print("the following threats have been targeted: {}".format(targetedThreats))
print("original threatValues: \n{}".format(originalVal))
print("average pK: {}".format(avgPk))
print("adjusted threat Vals: \n{}".format(threatVal))
print("biggest threat: {} with lethality: {}".format(bigDeal, bigLeth))

# MTSPOFS_GA Testbed

In [ ]:
timeStart = time.clock()

results = mtspofs_ga()

elapsedTime = time.clock() - timeStart

print("mtspofs_ga took {}s to complete".format(elapsedTime))

In [ ]:
print("optimalRoute: {}".format(results["optimalRoute"]))

In [ ]:
def rand_breaks(minTour, numPts, numBreaks, cumProb):
    if minTour == 1:     # No constraint on Breaks
        tmpBreaks = np.random.permutation( np.arange(numPts-1) )
        breaks = np.squeeze( np.sort(tmpBreaks[0:numBreaks]) )
    else:                # Force Breaks to be at Least the Minimum Tour Length
        testIndices = np.where(np.random.rand() < cumProb)[0]
        nAdjust = testIndices[0]
        spaces = np.ceil(numBreaks*np.random.rand(nAdjust))
        adjust = np.zeros(numBreaks)
        for kk in range(numBreaks):
            adjust[kk] = np.where(spaces == kk+1)[0].size
        # generate breaks array
        breaks = minTour*np.arange(numBreaks) + np.cumsum(adjust)
    # release results
    return breaks

def swapElements(vector, indices=np.array([])):
    # handle mis-typed indices agument
    subSet = np.squeeze( np.sort(np.asarray(indices, dtype=int)) )
    if subSet.size == 2:
        pos1 = subSet[0]
        pos2 = subSet[1]
        
        vocter = np.concatenate( 
                (vector[:pos1], [vector[pos2]], vector[pos1+1:pos2], 
                    [vector[pos1]], vector[pos2+1:]) 
                )
    else:
        # by default swapping just first and last element
        vocter = np.concatenate( 
                ([vector[-1]], vector[1:-1], [vector[0]]) 
                )

    # return transformed vector
    return vocter

def slideSet(vector, indices=np.array([]), shift=1):
    # handle mis-typed indices argument
    subSet = np.squeeze( np.sort(np.asarray(indices, dtype=int)) )
    shift = int( shift )
    if subSet.size == 2:
        stop1 = subSet[0]
        stop2 = subSet[1]

        vercto = np.concatenate( 
                (vector[:stop1], np.roll(vector[stop1:stop2], shift), vector[stop2:]) 
                )

    elif subSet.size == 1:
        stop = np.asscalar(subSet)
        vercto = np.concatenate( 
                (np.roll(vector[:stop], shift), np.roll(vector[stop:], shift)) 
                )

    else:
        vercto = np.roll( vector, shift )
        
    return vercto

In [ ]:
pointSet = 10*np.random.rand(40,2)
distMat = []
nSalesmen = 5
minTour = 2
populationSize = 80
iterationMax = 5e3
popGroupSize = 8

nMax = pointSet.shape[0]-1

# Initialization for the Route Break Point Selection
nBreaks = nSalesmen - 1
DoF = nMax - minTour*nSalesmen
tmpVec = np.ones(DoF+1)
for kk in range(1, nBreaks):
    tmpVec = np.cumsum(tmpVec)
# calculate cumulative probability
cumProb = np.cumsum(tmpVec)/sum(tmpVec)

# Initialize the populations
popRoute = np.zeros([populationSize, nMax])
popBreak = np.zeros([populationSize, nBreaks])
popRoute[0] = np.arange(nMax) + 1
popBreak[0] = rand_breaks(minTour, nMax, nBreaks, cumProb)
for kk in range(1, populationSize):
    popRoute[kk] = np.random.permutation( np.arange(nMax) ) + 1
    popBreak[kk] = rand_breaks(minTour, nMax, nBreaks, cumProb)

In [ ]:
randIndx = np.random.randint(pointSet.shape[0])

# routeInsertionPoints = np.squeeze( np.sort(np.ceil((nMax-1)*np.random.rand(1,2))) )
routeInsertionPoints = np.array([np.random.randint(popRoute.shape[1]-5), popRoute.shape[1]-1])
testRoute = slideSet(popRoute[randIndx], routeInsertionPoints, -1)

print("randIndx: {}, routeInsertionPoints: {}".format(randIndx, routeInsertionPoints))
print("original popRoute Vector: \n{}".format(popRoute[randIndx]))
print("mutated popRoute Vector: \n{}".format(testRoute))

print("\n size of original vector: {} \nsize of new vector: {}"\
     .format(popRoute[randIndx].size, testRoute.size))

In [ ]:
pointSet = 10*np.random.rand(40,2)
distMat = []
nSalesmen = 5
minTour = 2
populationSize = 80
iterationMax = 5e3
popGroupSize = 8

defaultConfig = {'pointSet': pointSet,
                'distMat': distMat,
                'nSalesmen': nSalesmen,
                'minTour': minTour,
                'populationSize': populationSize,
                'iterationMax': iterationMax,
                'popGroupSize': popGroupSize}

configuration = mods.SimpleNamespace(defaultConfig)

if configuration.populationSize % configuration.popGroupSize != 0:
    # adjust population size to be divisible by populationGroupSize
    newPopSize = configuration.populationSize + (configuration.popGroupSize - 
            configuration.populationSize % configuration.popGroupSize)
    # print warning message to alert operator of the change
    print("WARNING: populationSize {} is not divisible by popGroupSize {}. Padding populationSize to {}"\
            .format(configuration.populationSize, configuration.popGroupSize, newPopSize))
    # save the new populationSize to the configuration dictionary
    configuration.populationSize = newPopSize

pointSet = configuration.pointSet
distMat = configuration.distMat

if len(distMat) == 0:
    nPoints = pointSet.shape[0]
    distMat = np.zeros([nPoints, nPoints])
    for rowIndex in range(nPoints):
        for colIndex in range(nPoints):
            distMat[rowIndex, colIndex] = np.linalg.norm( pointSet[rowIndex] - pointSet[colIndex] )

# Verify inputs
if distMat.shape[0] != pointSet.shape[0] or distMat.shape[1] != pointSet.shape[0]:
    raise IndexError("provided pointSet and distMat variables are incompatible, shape {} cannot"
            + "be combined with shape {}".format(pointSet.shape, distMat.shape))
else:
    configuration.distMat = distMat

nMax = pointSet.shape[0] - 1

# Sanity checks
nSalesmen = int(configuration.nSalesmen)
minTour = int(configuration.minTour)
maxIterations = int(configuration.iterationMax)

# Initialization for the Route Break Point Selection
nBreaks = nSalesmen - 1
DoF = nMax - minTour*nSalesmen
tmpVec = np.ones(DoF+1)
for kk in range(1, nBreaks):
    tmpVec = np.cumsum(tmpVec)
# calculate cumulative probability
cumProb = np.cumsum(tmpVec)/sum(tmpVec)

# Initialize the populations
popRoute = np.zeros([configuration.populationSize, nMax])
popBreak = np.zeros([configuration.populationSize, nBreaks])
popRoute[0] = np.arange(nMax) + 1
popBreak[0] = rand_breaks(minTour, nMax, nBreaks, cumProb)
for kk in range(1, configuration.populationSize):
    popRoute[kk] = np.random.permutation( np.arange(nMax) ) + 1
    popBreak[kk] = rand_breaks(minTour, nMax, nBreaks, cumProb)

# Run the genetic algorithm
globalMin = np.inf
totalDistance = np.zeros(configuration.populationSize)
distanceHist = np.zeros(maxIterations)
tmpPopRoute = np.zeros([configuration.popGroupSize, nMax])
tmpPopBreak = np.zeros([configuration.popGroupSize, nBreaks])
newPopRoute = np.zeros([configuration.populationSize, nMax])
newPopBreak = np.zeros([configuration.populationSize, nBreaks])

for iteration in range(maxIterations):
    # Evaluate members of the Population
    for member in range(configuration.populationSize):
        distance = 0
        memRoute = popRoute[member]
        memBreak = popBreak[member]
        memRange = np.array([ np.concatenate(([0], memBreak+1)), np.concatenate((memBreak, [nMax]))], dtype=int)
        for index in range(configuration.nSalesmen):
            # find starting distance
            distance = distance + configuration.distMat[0, memRoute[memRange[0, index]]]
            # find the distance for the rest of the locations
            for kk in range(memRange[0, index], memRange[1, index]-1):
                distance = distance + configuration.distMat[memRoute[kk], memRoute[kk+1]]
        # store the distance information
        totalDistance[member] = distance

    # Find the best route to the population
    minIndex = totalDistance.argmin()
    minDist = totalDistance[ minIndex ]
    distanceHist[minIndex] = minDist
    if minDist < globalMin:
        globalMin = minDist
        optimalRoute = popRoute[minIndex]
        optimalBreak = popBreak[minIndex]
        popRange = np.array([np.concatenate(([0], optimalBreak+1)), 
            np.concatenate((optimalBreak, [nMax]))])
        
        print("optimalBreak: {} \noptimalRoute: {}".format(optimalBreak, optimalRoute))
        for salesman in range(nSalesmen):
            route = np.concatenate(([1], optimalRoute[popRange[0, salesman]:popRange[1, salesman]]))

print("chosen route: {}".format(route))
    
# print("total distance vector: {}".format(totalDistance))
# print("minimum distance measurement: {}".format(np.amin(totalDistance)))

In [ ]:
print(totalDistance.argmin())
totalDistance[totalDistance.argmin()]

# Classes Testbed

In [ ]:
def np2jsonDefault(obj):
    if isinstance(obj, np.ndarray):
        return obj.tolist()
    return obj.__dict__

def zipArray(arrays, axis=0):
    if isinstance(axis, int):
        try:
            shapes = [array.shape[0] for array in arrays]
            ndims  = [array.ndim for array in arrays]
            # zip will by default only combine up to the shortest array
            # currently this is unwanted so the ValueError exception is thrown
            if len(set(shapes)) > 1 or not np.array_equal(ndims, np.ones_like(ndims)):
                if not len(set(shapes)) > 1:
                    raise ValueError("all arrays must be 1 dimensional: {}".format(ndims))
                else:
                    raise ValueError("all arrays must have the same length: {}".format(shapes))
            if axis == 0:
                combinedArray = np.asarray( zip(*arrays) )
            else:
                combinedArray = np.squeeze( np.asarray( zip(arrays) ) )
            return combinedArray

        except IndexError:
            print("'arrays' argument must be a tuple or a list")
            return {"arrays": locals()["arrays"], "axis": locals()["axis"]}      
    else:
        raise TypeError("axis argument must be of type integer")

class lethalityMatrix(object):
    def __init__(self, pLethal, confidence, ranking):
        self.pLethal = pLethal
        self.confidence = confidence
        self.ranking = ranking

    def getLethality(self, index=None):
        index = np.asarray(index)
        if index.ndim == 0:
            return self.pLethal, self.confidence, self.ranking
        else:
            try:
                return self.pLethal[index], self.confidence[index], self.ranking[index]
            except IndexError:
                print("IndexError: the index, {}, is out of bounds for an array of length {}".format(index, 
                    pLethal.size))

class cvState(object):
    def __init__(self, rCV, vCV, rTracks, time):
        self.rCurr = rCV
        self.vCurr = vCV
        self.CVup = rCV
        self.time = time
        tmpVec = np.mean(rTracks, axis=0) - self.rCurr
        if np.linalg.norm(tmpVec) != 0:
            self.CVpointing = tmpVec / np.linalg.norm(tmpVec)
        else:
            self.CVpointing = tmpVec
        
    def extract(self, JSON=False):
        if not JSON:
            return self.__dict__
        else:
            return json.dumps(self.__dict__,  ensure_ascii=True, separators=(',', ': '), 
                    default=np2jsonDefault, indent=2)


class trackStates(object):
    def __init__(self, rTracks, vTracks, lethalityObj, **kwargs):
        self.rCurr = rTracks
        self.vCurr = vTracks
        # check for existing Ids field
        if 'Ids' in kwargs:
            self.Ids = kwargs["Ids"]
        else:
            self.Ids = np.arange(rTracks.shape[0])
        # check for individual lethalityMatrix fields
        if lethalityObj != None:
            self.pLethal, self.scpl, self.snr = lethalityObj.getLethality()
        else:
            self.pLethal = kwargs.get('pLethal')
            self.scpl = kwargs.get('scpl')
            self.snr = kwargs.get('snr')
        # auto generated fields
        self.active = np.ones_like(self.Ids, dtype=bool)
        self.collection = np.zeros((rTracks.shape[0], 11))

        self.collection[:, 0:6] = np.hstack((self.rCurr, self.vCurr))
        self.collection[:, 6:] = zipArray((self.Ids, self.active, self.pLethal, self.scpl, self.snr))

    def getTracks(self, index=None):
        index = np.asarray(index)
        if index.ndim == 0:
            return self.collection
        else:
            try:
                rawMat = self.collection[index]
                selectedTracks = trackStates(rawMat[:,0:3], rawMat[:,3:6], None, 
                        Ids=rawMat[:,6], pLethal=rawMat[:,8], scpl=rawMat[:,9], snr=rawMat[:,10])
                return selectedTracks
            except IndexError:
                print("IndexError: the index, {}, is out of bounds for an array of length {}".format(index, 
                    collection.shape[0]))

    def getActiveStates(self):
        index = self.active
        # index is a boolean array so we can use it to directly parse the values we want
        redTrackState = trackStates(self.rCurr[index], self.vCurr[index], None, 
                pLethal=self.pLethal[index], scpl=self.scpl[index], snr=self.snr[index], Ids=self.Ids[index])
        return redTrackState

    def extract(self, JSON=False):
        if not JSON:
            return self.__dict__
        else:
            return json.dumps(self.__dict__,  ensure_ascii=True, separators=(',', ': '), 
                    default=np2jsonDefault)


class simState(object):
    def __init__(self, time, **kwargs): 
        self.cvState = kwargs.get("cvState")
        self.trackStates = kwargs.get("trackStates")
        self.time = time
        if self.cvState == None:
            self.cvState = cvState(kwargs["rCV"], kwargs["vCV"], kwargs["rTracks"], time)
        if self.trackStates == None:
            self.trackStates = trackStates(kwargs["rTracks"], kwargs["vTracks"], kwargs["lethalityObject"])
    
    def extract(self, JSON=False):
        result = {"cvstate": self.cvState.extract(), "trackStates": self.trackStates.extract(), 
                "time": self.time}
        if not JSON:
            return result
        else:
            return json.dumps(result,  ensure_ascii=True, 
                    default=np2jsonDefault)

            
class stateCollection(object):
    def __init__(self, timeArray, **kwargs):
        print("input arguments recieved: {}".format(kwargs.keys()))
        self.state = [None] * timeArray.size
        print("lethalityObject: {}".format(kwargs.get('lethalityObject').getLethality()))
        print("empty state variable: {}".format(self.state))
        for count, step in enumerate(timeArray):
            self.state[count] = simState(step, **kwargs)
            
    def getState(self, time, unpack=False):
        stateIndex = [state.time for state in self.state].index(time)
        if not unpack:
            return self.state[stateIndex]
        else:
            return self.state[stateIndex].cvState, self.state[stateIndex].trackStates, self.state[stateIndex].time
        


In [3]:
nTracks = np.random.randint(10, 20)
pLethal = np.random.randint(10, size=nTracks)
scpl = np.random.rand(nTracks)
snr = np.random.rand(nTracks) * 30
lethalObject = lethalityMatrix(pLethal, scpl, snr)

rCVFinal = np.array([-7, 0, 0]) * 1e3
rMisl = np.zeros(3) * 1e3
vTrackTerm = np.array([0, -4.5, -5])
vCVTerm = np.array([0, 7.5, 0])
tFinal = 300

timeArray = np.linspace(0, tFinal, nTracks)

rCV0, vCV0 = Kepler(rCVFinal, -vCVTerm, tFinal)
vCV0 = -vCV0

rTgt0, vTgt0 = Kepler(rCVFinal-rMisl, -vTrackTerm, tFinal)
vTgt0 = -vTgt0

rDeviation = np.linalg.norm(rTgt0) * 0.01
vDeviation = np.linalg.norm(vTgt0) * 0.01

rTracks = np.tile(rTgt0, (nTracks, 1)) + rDeviation * np.random.randn(nTracks, 3)
vTracks = np.tile(vTgt0, (nTracks, 1)) + vDeviation * np.random.randn(nTracks, 3)

buildDict = {"rCV": rCV0, "vCV": vCV0, "rTracks": rTracks, "vTracks": vTracks, "lethalityObject": lethalObject,
            "timeArray": timeArray}

simStateTest = stateCollection(**buildDict)

randTime = timeArray[np.random.randint(timeArray.size)]

sampleState = simStateTest.getState(randTime)

print('location of time ({}) in simStateTest: {}'.format(randTime, [state.time for state in simStateTest.state].index(randTime)))
print('complete timeArray: \n{}'.format(zipArray((np.arange(timeArray.size), timeArray))))
print('resultant values \nrCV0: {} \nvCV0: {}'.format(rCV0, vCV0))
print('resultant simState piece: cvState {}'.format(simStateTest.state[np.random.randint(len(simStateTest.state))].cvState.rCurr))

ConvergenceError: exceeded specific number of iterations before convergence was achieved
ConvergenceError: exceeded specific number of iterations before convergence was achieved
location of time (125.0) in simStateTest: 5
complete timeArray: 
[[   0.    0.]
 [   1.   25.]
 [   2.   50.]
 [   3.   75.]
 [   4.  100.]
 [   5.  125.]
 [   6.  150.]
 [   7.  175.]
 [   8.  200.]
 [   9.  225.]
 [  10.  250.]
 [  11.  275.]
 [  12.  300.]]
resultant values 
rCV0: [ 0.  0.  0.] 
vCV0: [-0. -0. -0.]
resultant simState piece: cvState [ 0.  0.  0.]


In [4]:
pLethal = np.random.randint(10, size=nTracks)
scpl = np.random.rand(nTracks)
snr = np.random.rand(nTracks) * 30
lethalObject = lethalityMatrix(pLethal, scpl, snr)

print("pLethal: {}".format(lethalObject.pLethal))
print("StatConf of pLethal: {}".format(lethalObject.confidence))
print("snr vector: {}".format(lethalObject.ranking))
print(lethalObject.getLethality())

pLethal: [4 3 0 0 6 9 8 8 2 2 2 5 6]
StatConf of pLethal: [ 0.97282666  0.73355023  0.5095643   0.49369191  0.53495428  0.50771554
  0.13740049  0.84307499  0.87266264  0.75659151  0.4085008   0.79920058
  0.5241016 ]
snr vector: [ 14.01723033  29.54396559   6.24052803   6.97306493   2.93550432
  10.22231967  20.23103034   1.64519415   2.73715062   6.66890285
  25.08614254  23.99674726  27.29493609]
(array([4, 3, 0, 0, 6, 9, 8, 8, 2, 2, 2, 5, 6]), array([ 0.97282666,  0.73355023,  0.5095643 ,  0.49369191,  0.53495428,
        0.50771554,  0.13740049,  0.84307499,  0.87266264,  0.75659151,
        0.4085008 ,  0.79920058,  0.5241016 ]), array([ 14.01723033,  29.54396559,   6.24052803,   6.97306493,
         2.93550432,  10.22231967,  20.23103034,   1.64519415,
         2.73715062,   6.66890285,  25.08614254,  23.99674726,  27.29493609]))


In [5]:
testResult = sampleState.trackStates.rCurr

cvTest, trackTest, timeTest = simStateTest.getState(randTime, unpack=True)

print("direct test result: {}".format(simStateTest.state[0].extract(JSON=True)))
print("unpacked result, \ncvState: {}".format(cvTest.extract(JSON=True)))
print("unpacked result, trState: {}".format(trackTest.__dict__))
print("unpacked result, time: {}".format(timeTest))

print("\n linspace timeArray: {}".format(np.linspace(0, tFinal, nTracks)))

direct test result: {"trackStates": {"vCurr": [[-0.0,-0.0,0.0],[-0.0,-0.0,-0.0],[0.0,-0.0,0.0],[-0.0,0.0,0.0],[0.0,0.0,0.0],[0.0,-0.0,-0.0],[-0.0,-0.0,0.0],[-0.0,-0.0,0.0],[0.0,0.0,0.0],[-0.0,-0.0,0.0],[0.0,-0.0,-0.0],[-0.0,0.0,0.0],[-0.0,0.0,-0.0]],"pLethal": [5,2,4,1,3,0,1,1,4,9,3,8,1],"Ids": [0,1,2,3,4,5,6,7,8,9,10,11,12],"collection": [[0.0,0.0,0.0,-0.0,-0.0,0.0,0.0,1.0,5.0,0.8933496142472291,16.510020032906535],[0.0,0.0,0.0,-0.0,-0.0,-0.0,1.0,1.0,2.0,0.025199469661194462,8.925823631825041],[0.0,0.0,0.0,0.0,-0.0,0.0,2.0,1.0,4.0,0.5416170718561542,15.52824290761109],[0.0,0.0,0.0,-0.0,0.0,0.0,3.0,1.0,1.0,0.12026549331094316,13.10240092288206],[0.0,0.0,0.0,0.0,0.0,0.0,4.0,1.0,3.0,0.22894442711262142,24.96506180767354],[0.0,0.0,0.0,0.0,-0.0,-0.0,5.0,1.0,0.0,0.32413512386656795,22.1289239988661],[0.0,0.0,0.0,-0.0,-0.0,0.0,6.0,1.0,1.0,0.48913877822293694,23.97685853889939],[0.0,0.0,0.0,-0.0,-0.0,0.0,7.0,1.0,1.0,0.39122724698871447,21.07125669595009],[0.0,0.0,0.0,0.0,0.0,0.0,8.0,1.0,4.0,0

In [10]:
trackTest.tslv = np.ones(len(trackTest))

print("additional vector added to trackTest: {}".format(trackTest.tslv))
print("updated TrackTest variable: \n{}".format(trackTest.extract()))

additional vector added to trackTest: [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
updated TrackTest variable: 
{'vCurr': array([[-0., -0.,  0.],
       [-0., -0., -0.],
       [ 0., -0.,  0.],
       [-0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0., -0., -0.],
       [-0., -0.,  0.],
       [-0., -0.,  0.],
       [ 0.,  0.,  0.],
       [-0., -0.,  0.],
       [ 0., -0., -0.],
       [-0.,  0.,  0.],
       [-0.,  0., -0.]]), 'pLethal': array([5, 2, 4, 1, 3, 0, 1, 1, 4, 9, 3, 8, 1]), 'Ids': array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12]), 'collection': array([[  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         -0.00000000e+00,  -0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   1.00000000e+00,   5.00000000e+00,
          8.93349614e-01,   1.65100200e+01],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         -0.00000000e+00,  -0.00000000e+00,  -0.00000000e+00,
          1.00000000e+00,   1.00000000e+00,   2.00000000e+0

In [ ]:
testTracks = np.unique( np.random.randint(nTracks, size=(nTracks/2)) )

unVisitedTrackIndex = []
for goodId in testTracks:
    indices = np.where(sampleState.trackStates.Ids == goodId)[0]
    unVisitedTrackIndex.append(indices[0])

selectedTracks = sampleState.trackStates.getTracks(unVisitedTrackIndex)
print("sampleState trackIds: {}".format(sampleState.trackStates.Ids))
print("test tracks: {}".format(testTracks))
print("unvisitedTrackIndex: {}".format(selectedTracks.rCurr))

In [ ]:
randomActivity = np.random.randint(2, size=nTracks).astype(bool)

simStateTest.getState(randTime).trackStates.active = randomActivity

copyOfTracks = sampleState.trackStates.copy()

print("random activity vector: {}".format(randomActivity))
print("full trackStates for this selection: {}".format(simStateTest.getState(randTime).trackStates.Ids))
print("active states from this selection: {}".format(simStateTest.getState(randTime).trackStates.getActiveStates().Ids))

# Integration Test

In [23]:
def PlanckBB(wavelength, temp):
    K = physMath.BOLTZMAN
    c = physMath.SPEED_LIGHT
    h = physMath.PLANCK

    exitance = 2.0*np.pi*h*np.square(c) / (np.sqrt(wavelength)*np.expm1(h*c/(wavelength*K*temp)))

    return exitance

def calculateSNR(slantRange, features, aspect=np.pi/4):
    # constants
    T_atm = 1.0 # assuming exoatmospheric observation (no absorption)
    SKR_BAND1 = np.array([3.0, 5.0]) * 1e-6
    SKR_BAND2 = np.array([8.0, 12.0]) * 1e-6

    # find the target blackbody radiance over sensor wavelength (W/m^2/sr)
    def getPoints(lower, upper, precision):
        return np.linspace(lower, upper+1, precision)

    mwirDomain = getPoints(SKR_BAND1[0], SKR_BAND1[1], 1000)
    lwirDomain = getPoints(SKR_BAND2[0], SKR_BAND2[1], 1000)

    mwirRange = PlanckBB(mwirDomain, features.get("temperature"))
    lwirRange = PlanckBB(lwirDomain, features.get("temperature"))

    mwirRadiance = np.trapz(mwirRange, mwirDomain)
    lwirRadiance = np.trapz(lwirRange, lwirDomain)
    
    return mwirRadiance, lwirRadiance

In [24]:
feature = {"temperature": 300}

mwirRadiance, lwirRadiance = calculateSNR(0.0, feature)

print("mwir Radiance: {}".format(mwirRadiance))
print("lwir Radiance: {}".format(lwirRadiance))

mwir Radiance: 5.20099430988e-12
lwir Radiance: 5.20104822079e-12


# ModMunkres Testbed

In [ ]:
def buildStarsAndPrimes(sourceMat):
    # now lets do some fancy matrix math!
    # find the zeros of the matrix:
    rowZeros, colZeros = np.where(sourceMat == 0)
    rowUniques = np.unique(rowZeros)
    combinedZeros = np.asarray( zip(rowZeros, colZeros) )
    starsCon = np.zeros_like(combinedZeros[:,0], dtype=bool)
    #------------------------- STEP 2 & STEP 3 --------------------------
    coveredCols = []
    coveredRows = []
    for val in rowUniques:
        subArray = np.where(combinedZeros[:,0] == val)[0]
        shortList = combinedZeros[subArray][:,1]
        for column in shortList:
            if np.in1d(column, coveredCols).any():
                # no good, skip to the next column
                continue
            else:
                subMatLow = ( (combinedZeros[:,0] == val) & (combinedZeros[:,1] == column) )
                coveredCols.append(column)
                starsCon = np.logical_or(starsCon, subMatLow)
                break
    # do some minor array manipulation for clarity
    stars = combinedZeros[starsCon]                             # STEP 2
    primesCandidate = combinedZeros[~starsCon]                  # STEP 3
    primesConfirmed = []
    for index, pair in enumerate(primesCandidate):
        if np.in1d(pair[0], coveredRows).any() or np.in1d(pair[1], coveredCols).any():
            # this zero is already covered
            continue
        else:
            primesConfirmed.append(index)
            coveredRows.append(pair[0])
            openRow = np.where(stars[:,0] == pair[0])[0]
            try:
                coveredCols.remove(stars[openRow[0]][1])
            except:
                # the column was not covered initially anyway
                continue
            
    # get final list of primes (uncovered zeros)
    primes = primesCandidate[primesConfirmed]

    # return the new stuff:
#     return {"stars": stars,"primes": primes,"combinedZeros": combinedZeros,
#             "coveredRows": coveredRows,"coveredCols": coveredCols}
    return stars, primes, {"coveredRows": coveredRows,"coveredCols": coveredCols}

def refactorPreliminaries(sourceMat, coveredRows, coveredCols):
    #------------------------- STEP 5 --------------------------
    originalMatrix = sourceMat.copy()
    adjustedMatrix = sourceMat.copy()
    # figure our what is uncovered
    unCoveredRows = np.setxor1d( coveredRows, np.arange(sourceMat.shape[0]) )
    unCoveredCols = np.setxor1d( coveredCols, colRange )
    # calculate what is left
    reducedMat = np.delete(adjustedMatrix, coveredRows, axis=0)
    reducedMat = np.delete(reducedMat, coveredCols, axis=1)
    # find the smallest remaining value (should be non-zero)
    if reducedMat.size == 0 or unCoveredCols.size == 0:
        pass
    else:
        absMin = np.amin(reducedMat)
        # adjust the preliminary matrix with the new information
        adjustedMatrix[coveredRows, :] = adjustedMatrix[coveredRows, :] + absMin
        adjustedMatrix[:, unCoveredCols] = adjustedMatrix[:, unCoveredCols] - absMin 
    # return the adjusted matrix and original matrix:
    return adjustedMatrix, originalMatrix

def calculateFinalResult(stars, sourceMat):
    # this will exectute when the stars matrix finally satisfies the desired min condition
    finalPerm = stars[:,1]
    permSum = 0.0
    for pair in stars:
        permSum += sourceMat[pair[0]][pair[1]]

    # return the results
    return finalPerm, permSum

In [ ]:
sampleMat = np.random.randint(10, size=(10,10))

numRows, numCols = sampleMat.shape
rowRange = np.arange(numRows)
colRange = np.arange(numCols)
finalPerm = []
permSum = 0

print("sampleMat: \n{} \n shape: {}".format(sampleMat, sampleMat.shape))

In [ ]:
prelimMat = sampleMat.copy()
incomplete = True
maxCount = 1000
iteration = 0

if numRows >= numCols:
    minval = np.apply_along_axis(np.amin, 0, prelimMat)
    prelimMat = prelimMat - minval
# NOTE: both if statements will execute in the event numCols = numRows
if numRows <= numCols:
    minval = np.reshape( np.apply_along_axis(np.amin, 1, prelimMat), (numRows, 1) )
    prelimMat = prelimMat - minval
    if numRows < numCols:
        perlimMat = np.transpose(prelimMat)
        
stars, primes, coverings = buildStarsAndPrimes(prelimMat)

starsUnique = np.unique(stars[:,1])

if np.array_equal(starsUnique, colRange):
    finalPerm, permSum = calculateFinalResult(stars, sampleMat)
else:
    while incomplete:
        prelimMat, originalMat = refactorPreliminaries(prelimMat, **coverings)
        stars, primes, coverings = buildStarsAndPrimes(prelimMat)
        
        starsUnique = np.unique(stars[:,1])
        if np.array_equal(starsUnique, colRange) or iteration == maxCount:
            finalPerm, permSum = calculateFinalResult(stars, sampleMat)
        else:
            iteration += 1
            continue

print("finalPerm: {}, permSum: {}".format(finalPerm, permSum))

In [ ]:
permCheck1, permCheck2, sumCheck = ModMunkres2(sampleMat)

print("finalPerm from code: {}, permSum from code: {}".format(permCheck1, sumCheck))

## Rigorous Integer Testing

In [ ]:
start = time.clock()
for ii in range(1000):
    rowSize = np.random.randint(3,10)
    colSize = np.random.randint(3,10)
    sampleMat = np.random.randint(10, size=(rowSize,colSize))
    permCheck1, permCheck2, sumCheck, booleanMask = ModMunkres2(sampleMat, int)
    elapsedTime = (time.clock() - start)
    if ii % 100 == 0:
        print("step {} - ".format(ii))
        print("elapsed time: {}s".format(elapsedTime))
        print("shape of sample Mat: {}".format(sampleMat.shape))
        print("finalRows: {} \nfinalCols: {} \npermSum from code: {}\n".format(permCheck1, permCheck2, sumCheck))
        print("boolean Mask: \n{}".format(booleanMask))

In [ ]:
print("integer form of booleanMask: {}".format(booleanMask.astype(int)))

## Rigorous Decimal Testing

In [ ]:
start = time.clock()
for ii in range(1000):
    rowSize = np.random.randint(3,10)
    colSize = np.random.randint(3,10)
    sampleMat = np.random.rand(rowSize,colSize) * 10
    permCheck, sumCheck = ModMunkres2(sampleMat)
    elapsedTime = (time.clock() - start)
    if ii % 100 == 0:
        print("step {} - ".format(ii))
        print("elapsed time: {}s".format(elapsedTime))
        print("shape of sample Mat: {}".format(sampleMat.shape))
        print("finalPerm from code: {}, permSum from code: {}\n".format(permCheck, sumCheck))